In [51]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load the data
data = pd.read_csv('CycleData.csv')

# Select the relevant columns
selected_columns = ['Age', 'LengthofCycle', 'LengthofMenses', 'LengthofLutealPhase', 
                    'TotalNumberofHighDays', 'TotalNumberofPeakDays', 'TotalDaysofFertility', 
                    'TotalFertilityFormula', 'BMI', 'EstimatedDayofOvulation']
data_selected = data[selected_columns]

# Replace empty spaces with NaN, convert to float, and fill NaNs with the median
for column in ['Age', 'LengthofMenses', 'LengthofLutealPhase', 'TotalNumberofHighDays', 
               'TotalNumberofPeakDays', 'TotalDaysofFertility', 'TotalFertilityFormula', 'BMI']:
    data_selected[column] = data_selected[column].replace(' ', np.nan)
    data_selected[column] = data_selected[column].astype(float)
    data_selected[column].fillna(data_selected[column].median(), inplace=True)

# Convert 'LengthofCycle' to float (it's already numeric and doesn't contain any missing values)
data_selected['LengthofCycle'] = data_selected['LengthofCycle'].astype(float)


C:\Users\offro\AppData\Local\Temp\ipykernel_22936\430425513.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected[column] = data_selected[column].replace(' ', np.nan)
C:\Users\offro\AppData\Local\Temp\ipykernel_22936\430425513.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected[column] = data_selected[column].astype(float)
C:\Users\offro\AppData\Local\Temp\ipykernel_22936\430425513.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame



In [ ]:
data_selected.to_csv('Cleaned_CycleData_selected.csv', index=False)

In [44]:

# Convert 'EstimatedDayofOvulation' to float, and drop rows with missing values in this column
data_selected['EstimatedDayofOvulation'] = data_selected['EstimatedDayofOvulation'].replace(' ', np.nan).astype(float)
data_selected = data_selected.dropna(subset=['EstimatedDayofOvulation'])

# Separate the data into features (X) and target (y)
X = data_selected.drop(columns='EstimatedDayofOvulation')
y = data_selected['EstimatedDayofOvulation']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Determine the number of components to keep
explained_variance_ratio = pca.explained_variance_ratio_.cumsum()
n_components = (explained_variance_ratio < 0.95).sum() + 1

# Re-apply PCA with the optimal number of components
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

# Show the shape of the data after PCA
X_pca.shape


C:\Users\offro\AppData\Local\Temp\ipykernel_22936\1767525346.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected[column] = data_selected[column].replace(' ', np.nan)
C:\Users\offro\AppData\Local\Temp\ipykernel_22936\1767525346.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected[column] = data_selected[column].astype(float)
C:\Users\offro\AppData\Local\Temp\ipykernel_22936\1767525346.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

(1515, 7)

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

train_score, test_score


(0.8982347915484865, 0.8817201741783073)

In [47]:
# Select the first sample from the test set
sample_X = X_test[0].reshape(1, -1)  # Reshape because the model expects a 2D array
sample_y = y_test.iloc[0]

# Make a prediction for the sample
prediction = model.predict(sample_X)

# Show the true and predicted values
sample_y, prediction[0]


(17.0, 17.04973417131074)

In [49]:
sample_X

17.0

In [ ]:
sample_y


In [50]:
prediction

array([17.04973417])